In [10]:
from datetime import datetime
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from dateutil.parser import parse
from sklearn.metrics import silhouette_score

ProgressBar().register()

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.cluster import KMeans

In [14]:
taxi_zones = gpd.read_file('data/taxi_zones__7_/taxi_zones.shp')

In [15]:
taxi_zones

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824...."


# ACS and subway ridership data

In [59]:
# Import pct_change in subway ridership from May through August between 2019 and 2020

subway = pd.read_csv('MLC_data-main 3/data/ridership_by_taxizone/ridership_by_taxizone.csv')
subway = subway.drop(columns=['Unnamed: 0', '2019_entries', '2019_exits','2020_entries','2020_exits'])
subway['mean_pct_change'] = (subway['entries__pct_change'] + subway['exits_pct_change']) / 2

In [60]:
subway['subway'] = 1
subway['subway'] = subway['subway'].mask(subway.isna().any(axis=1), 0)
subway = subway.drop(columns=['entries__pct_change','exits_pct_change','mean_pct_change'])

In [16]:
# Import 2019 ACS variables for census_tracts and taxi

acs = pd.read_csv('MLC_data-main 3/data/census_zones/census_zones.csv')

In [27]:
# select relevant variables from ACS table

acs_var_names = {'healthcare':'SE_B17008_004','food_service':'SE_B17008_006','maintenance':'SE_B17008_007',\
                 'retail_sales':'SE_B17008_009', 'race_white':'SE_A03001_002', 'hh_median_income':'SE_A14006_001',\
                 'transit_commute':'SE_A09005_003','transit_12_5':'ACS19_5yr_B08132047','transit_60min+':'ACS19_5yr_B08134070'}

acs_var_names = {v: k for k, v in acs_var_names.items()}

acs_vars = [s.strip() for s in list(acs_var_names.keys())]

In [18]:
acs_vars = pd.DataFrame.from_dict(acs_var_names, orient='index', columns=['Variable'])
acs_vars.style.set_caption('American Community Survey Variables')

,Variable
SE_B17008_004,healthcare
SE_B17008_006,food_service
SE_B17008_007,maintenance
SE_B17008_009,retail_sales
SE_A03001_002,race_white
SE_A14006_001,hh_median_income
SE_A09005_003,transit_commute
ACS19_5yr_B08132047,transit_12_5
ACS19_5yr_B08134070,transit_60min+


In [29]:
zone_acs1 = acs[['LocationID'] + acs_vars].groupby('LocationID').sum().drop(columns='SE_A14006_001')
zone_acs2 = acs[['LocationID','SE_A14006_001']].groupby('LocationID').mean()
zone_acs = zone_acs1.merge(zone_acs2, on='LocationID').fillna(0)
zone_acs.drop(columns='Unnamed: 0')

KeyError: "['Unnamed: 0'] not found in axis"

In [31]:
zone_acs.columns = [acs_var_names[x] for x in zone_acs.columns]

In [32]:
zone_acs['essential_workers'] = zone_acs.iloc[:,:4].sum(axis=1)

In [33]:
zone_acs = zone_acs.reset_index().drop(columns=['healthcare','food_service','maintenance','retail_sales'])

In [34]:
zone_acs = zone_acs.iloc[1:].reset_index(drop=True)

In [35]:
scaler = StandardScaler()
scaled_acs = scaler.fit_transform(zone_acs.iloc[:,1:])

In [36]:
scaled_acs

array([[-0.38213346, -0.40778436, -0.13403232,  0.2479907 ,  0.09056315,
        -0.23237315],
       [-0.3092298 , -0.19485361, -0.27737052, -0.64787868, -0.60345385,
        -0.32133768],
       [ 0.85407319, -0.82856973, -0.74321966, -0.28946374,  0.37816549,
        -0.48260448],
       ...,
       [-0.87651903, -0.92194401, -0.73605275, -1.01066043,  2.19846468,
        -1.07708986],
       [ 1.21276569,  0.62921781, -0.43504254, -0.67441549,  0.88918564,
        -0.27789681],
       [ 1.68328158,  1.50939394, -0.68230092, -0.51754597,  0.96247209,
         0.02767701]])

# ACS Principal Component Analysis

In [37]:
#perform PC decomposition over Scaled ACS
pca = PCA(1)
p_acs =pca.fit_transform(scaled_acs)
eigenvalues = pca.explained_variance_ratio_

array([[-3.57534848e-01],
       [-7.03560401e-01],
       [-9.48264623e-01],
       [-1.38546338e+00],
       [ 4.25510158e+00],
       [-2.14719331e+00],
       [-1.29749339e+00],
       [ 2.90097896e-01],
       [ 2.81555337e-01],
       [-2.14719331e+00],
       [-1.84268811e+00],
       [ 3.63610330e+00],
       [-1.26787111e+00],
       [-1.78558505e-03],
       [ 2.57200292e+00],
       [ 3.28917722e+00],
       [-9.64353611e-01],
       [-2.88137823e-01],
       [ 3.09109501e+00],
       [ 4.83196997e+00],
       [-2.24214286e-01],
       [-1.34340853e+00],
       [-8.93023685e-01],
       [ 2.33894604e+00],
       [-2.29930036e+00],
       [ 1.09785421e+00],
       [ 4.73938410e-01],
       [-2.22188582e+00],
       [-2.14656272e+00],
       [ 1.12414964e+00],
       [-1.15749693e+00],
       [-2.43454516e+00],
       [ 1.29362477e+00],
       [ 2.22021417e+00],
       [ 3.99469725e+00],
       [-1.09081251e+00],
       [ 5.59488223e+00],
       [-1.45017025e+00],
       [ 1.0

In [38]:
P_acs = pd.DataFrame(zone_acs['LocationID']).join(pd.DataFrame(p_acs)).rename(columns={0:'PCA'})

In [39]:
P_acs.to_csv('FHV_ACS_principal_component.csv', index=False)

In [ ]:
subway = pd.read_csv('data/ridership_by_taxizone.csv')

In [ ]:
subway['subway'] = 1
subway['subway'] = subway['subway'].mask(subway.isna().any(axis=1), 0)

In [ ]:
bus_stops = pd.read_csv('data/FHV_bus_stops.csv')

In [ ]:
zone_acs = zone_acs.merge(subway[['LocationID','subway']], on='LocationID', how='left' )\
    .merge(bus_stops, on='LocationID', how='left')

In [50]:
scaler = StandardScaler()
scaled_acs = scaler.fit_transform(zone_acs.iloc[:,1:-2])

ValueError: Found array with 0 sample(s) (shape=(0, 772)) while a minimum of 1 is required by StandardScaler.

In [327]:
scaled_acs = pd.DataFrame(zone_acs['LocationID']).join(pd.DataFrame(scaled_acs))

In [330]:
scaled_acs = scaled_acs.merge(subway[['LocationID','subway']], on='LocationID', how='left' )\
    .merge(bus_stops, on='LocationID', how='left')

In [ ]:
scaled_acs

# ACS Clustering

In [332]:
for n_clusters in range(2,25):
    km = KMeans(random_state=1234 ,n_clusters=n_clusters)
    res= km.fit((scaled_acs))   
    cluster_labels = res.labels_
    silhouette_avg = silhouette_score(scaled_acs, cluster_labels)
    print("For n_clusters = {},".format(n_clusters)+" the average silhouette_score is : {}".format(silhouette_avg))

For n_clusters = 2, the average silhouette_score is : 0.6216196502210057
For n_clusters = 3, the average silhouette_score is : 0.5808646041018086
For n_clusters = 4, the average silhouette_score is : 0.5553592520671685
For n_clusters = 5, the average silhouette_score is : 0.5463160888480089
For n_clusters = 6, the average silhouette_score is : 0.5291211701722299
For n_clusters = 7, the average silhouette_score is : 0.5186346375944999
For n_clusters = 8, the average silhouette_score is : 0.5078956991597525
For n_clusters = 9, the average silhouette_score is : 0.49428666420919665
For n_clusters = 10, the average silhouette_score is : 0.4902514213834217
For n_clusters = 11, the average silhouette_score is : 0.48296091881852415
For n_clusters = 12, the average silhouette_score is : 0.4735066442018822
For n_clusters = 13, the average silhouette_score is : 0.4638108848099379
For n_clusters = 14, the average silhouette_score is : 0.45444329007663514
For n_clusters = 15, the average silhouette

In [ ]:
zone_acs.style.set_caption('')

In [344]:
km = KMeans(random_state=1234 ,n_clusters=2)
res= km.fit(scaled_acs)   

zone_acs['cluster'] = res.labels_
#zone_acs['cluster'] = [2] + list(res.labels_)

In [345]:
zone_acs

,LocationID,race_white,transit_commute,transit_12_5,transit_60min+,hh_median_income,essential_workers,subway,bus_stop_count,cluster
0,3.0,9220,5752,198,3922,78678.545455,3205,1,0,1
1,4.0,10121,7346,158,1255,48225.166667,2906,1,0,1
2,5.0,24498,2602,28,2322,91298.500000,2364,0,0,1
3,6.0,11378,2640,36,1767,78611.000000,1490,0,0,1
4,7.0,46868,32033,676,6049,73046.652174,10137,1,0,1
...,...,...,...,...,...,...,...,...,...,...
253,259.0,10283,10235,473,6385,70306.000000,5568,1,0,0
254,260.0,18985,15002,385,2996,64765.833333,6166,1,0,0
255,261.0,3110,1903,30,175,171173.000000,366,1,0,0
256,262.0,28931,13515,114,1176,113722.000000,3052,0,0,0


In [346]:
ACS_clusters = zone_acs.iloc[:,1:].groupby('cluster').mean()

In [347]:
ACS_clusters

,race_white,transit_commute,transit_12_5,transit_60min+,hh_median_income,essential_workers,subway,bus_stop_count
cluster,,,,,,,,
0,14220.325758,8328.719697,215.318182,2934.977273,79010.909058,3682.810606,0.848485,0.272727
1,13651.857143,9303.293651,256.444444,3444.333333,70193.322153,4303.587302,0.825397,0.396825


In [348]:
zone_acs.to_csv('ACS_taxizone_cluster.csv', index=False)